In [1]:
import os
import gc
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image
import glob
from sklearn import model_selection
from IPython.display import clear_output
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
names = ['lesion_id','image_id','dx','dx_type','age','sex','localization']
image_list = []
i=0
a=0

folder = "D:/Datasets/ResizedImage"
print(folder)
path, dirs, files = next(os.walk(folder))
file_count = len(files)
print(file_count)
pic_folder = sorted(glob.glob(folder + '/*.jpg'))
tenth = file_count/10
counter = tenth
for filename in pic_folder:
    img = Image.open(filename)
    data = np.asarray(img, dtype="int32" )
    image_list.append(data)
    if i>counter:
        #clear_output()
        counter = counter+tenth
        print(counter/file_count,'% done loading images')
    i = i +1
del pic_folder 
gc.collect()

Num GPUs Available:  1
D:/Datasets/ResizedImage
10015
0.2 % done loading images
0.3 % done loading images
0.4 % done loading images
0.5 % done loading images
0.6 % done loading images
0.7 % done loading images
0.8 % done loading images
0.9 % done loading images
1.0 % done loading images


0

In [2]:
dataset = pd.read_csv('D:/Datasets/HAM10000_metadata(Mel).csv', dtype=None, names=names) 
new_df = dataset.sort_values(by=["image_id"]) 
del dataset 
print("Array Sorted") 
X = np.array(image_list) 
print("X Array Created") 
del image_list 
last = new_df.shape 
Y = new_df.values[1:X.shape[0]-1,2] 
Y = Y.astype(np.int64) 
print(type(Y[0])) 
print("Y Array Created") 
X = X[:Y.shape[0]] 
gc.collect() 
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X[:Y.shape[0]], Y, test_size=.2, random_state=5) 
del X 
del Y 
print("Test/Train Sets Created")

Array Sorted
X Array Created
<class 'numpy.int64'>
Y Array Created
Test/Train Sets Created


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(90, 120,3)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(.2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    #tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])


Found GPU at: /device:GPU:0


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 88, 118, 32)       896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 86, 116, 32)       9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 56, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 39, 54, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 27, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 32832)             0

In [6]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=5)
model.evaluate(X_validation, Y_validation)

Train on 8010 samples
Epoch 1/5
6368/8010 [======================>.......] - ETA: 3s - loss: nan - accuracy: 0.1131

KeyboardInterrupt: 